In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

In [ ]:
from google.cloud import firestore

db = firestore.Client()

users_ref = db.collection("userData")
users = users_ref.stream()

updated = 0
skipped = 0

for user_doc in users:
    user_data = user_doc.to_dict()

    # Only judges
    if user_data.get("role") != "judge":
        skipped += 1
        continue

    event_ref = user_data.get("eventRef")
    if not event_ref:
        print(f"⚠️ Skipping {user_doc.id}: no eventRef")
        skipped += 1
        continue

    # ✅ FIX: eventRef is a DocumentReference
    event_code = event_ref.id

    event_doc = db.collection("eventJudgeMapping").document(event_code).get()
    if not event_doc.exists:
        print(f"⚠️ Skipping {user_doc.id}: eventJudgeMapping missing for {event_code}")
        skipped += 1
        continue

    judge_order = event_doc.to_dict().get("judgeOrder", [])
    if not judge_order:
        print(f"⚠️ Skipping {user_doc.id}: no judges found")
        skipped += 1
        continue

    judge_name = judge_order[0]["name"]  # first judge

    # Update ONLY name
    users_ref.document(user_doc.id).update({
        "name": judge_name
    })

    print(f"✅ Updated {user_doc.id} → {judge_name}")
    updated += 1

print("\n🎯 DONE")
print("Updated:", updated)
print("Skipped:", skipped)
